# Fine Tuning em Hiperparâmetros

Os hiperparâmetros são valores que são definidos pelo(s) criador(es) do modelo para que ele funcione da forma esperada. Não há um valor pré-definido que irá funcionar perfeitamente para um modelo qualquer, então é preciso ajustar um valor que obterá uma performance melhor para o modelo. Esse processo é conhecido como fine tuning.

O processo depende que alguns valores sejam testados no modelo e que a performance do modelo seja avaliada constantemente até que um valor razoável seja encontrado para o hiperparâmetro.

Para esse modelo, os hiperparâmetros que passarão pelo processo de fine tuning são:

- número de épocas (epoch)
- taxa de aprendizado (learning rate)
- proporção dos passos de aquecimento (warm up proportion)
- palavras de introdução
- taxa de decaimento (weight decay)

## Fine tuning para 'Epoch'

O processo de fine tuning para o hiperparâmetro de epoch será realizado de forma isolada. Isso significa que valores para os outros hiperparâmetros permanecerão fixos, enquanto o modelo é treinado para um número 'n' de epochs. Por meio dos passos de validação, durante o treinamento do modelo, é possível verificar a performance do modelo enquanto ele é treinado ao longo das épocas.

Configuração:

```
    'BATCH_SIZE': 2,
    'MAX_NUMBER_TOKENS': 512,
    'NUMBER_OF_BRANCHES': 13,
    'EPOCHS': 5,
    'LEARNING_RATE': __,
    'WEIGHT_DECAY': __,
    'WARM_UP_PROPORTION': __,
    'INTRODUCTION_WORDS': __,
```

## Fine tuning para 'Taxa de Aprendizado', 'Taxa de Decaimento', 'Palavras de introdução' e 'Proporção de passos de aquecimento'

Para definir os valores dos hiperparâmetros 'taxa de aprendizado', 'taxa de aquecimento', 'proporção de passos de aquecimento' e 'palavras de introdução', alguns valores foram escolhidos, de acordo com as sugestões [desse paper](https://arxiv.org/pdf/1905.05583.pdf). Dessa forma, todas as combinações de cada um dos valores que cada hiperparâmetro pode assumir será testado:

- Taxa de aprendizado: 2e-5, 3e-5, 5e-5
- Taxa de decaimento: 0.001, 0.01
- Palavras de introdução: 256, 384
- Proporção de passos de aquecimento: 0.1, 0.3


## Inicialização e definiçao de constantes

Como uma etapa inicial, toda a inicialização do notebook será concentrada no início desse documento. Os conteúdos contidos aqui são:

1. Instalação de bibliotecas externas
2. Importação de biblioteca
3. Definição de valores constantes que podem ter seu uso replicado ao longo do notebook
4. Inicialização do sistema de arquivos integrado ao Google Drive

Ao fim dessa seção, os hiperparâmetros são definidos.

In [ ]:
# Installation of 3rd party libraries

!pip install transformers
!pip install --upgrade pytorch-lightning

     |████████████████████████████████| 2.9 MB 5.5 MB/s 
     |████████████████████████████████| 3.3 MB 30.9 MB/s 
     |████████████████████████████████| 895 kB 37.3 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 39.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 925 kB 4.1 MB/s 
     |████████████████████████████████| 829 kB 35.6 MB/s 
     |████████████████████████████████| 125 kB 38.1 MB/s 
     |████████████████████████████████| 282 kB 42.0 MB/s 
     |████████████████████████████████| 1.3 MB 39.2 MB/s 
     |████████████████████████████████| 271 kB 41.4 MB/s 
     |████████████████████████████████| 160 kB 39.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=78bd2573574f9df1ac4e68d837fef8ce5ae7a0755db535f3409b322bc6336b53
  Store

In [ ]:
# Imports

from google.colab import drive
import pandas as pd
import numpy as np
from transformers import BertTokenizerFast as BertTokenizer, BertForSequenceClassification, AdamW, get_cosine_schedule_with_warmup, Trainer, TrainingArguments
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import re
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, matthews_corrcoef
from typing import List

In [ ]:
# Constants

CONSTANTS = {
    'TRAINING_DATASET': '/content/drive/My Drive/MAC499 - Kaique e Yurick/DB/Train_Dataset.csv',
    'VALIDATION_DATASET': '/content/drive/My Drive/MAC499 - Kaique e Yurick/DB/Validation_Dataset.csv',
    'BERT_MODEL_NAME': 'neuralmind/bert-large-portuguese-cased',
    'SEED': 13
}

# Hyperparameters

HYPERPARAMETERS = {
    'BATCH_SIZE': 2,
    'MAX_NUMBER_TOKENS': 512,
    'NUMBER_OF_BRANCHES': 13,
    'EPOCHS': 5,
    'LEARNING_RATE': 2e-5,
    'WEIGHT_DECAY': 0.001,
    'WARM_UP_PROPORTION': 0.1,
    'INTRODUCTION_WORDS': 256
}

In [ ]:

# Mounting Google Drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Verificar disponibilidade da GPU

O próximo passo seria verificar se a GPU oferida pela Google gratuitamente como ambiente de execução do notebook está funcionando corretamente. A GPU oferece uma performance computacional maior em relação a calculos sendo executados pela CPU.

In [ ]:
torch.cuda.empty_cache()

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


### Reproducibilidade

Para fins de reproducibilidade, definimos uma semente para o pytorch lightning.

In [ ]:
seed_everything(CONSTANTS['SEED'])

Global seed set to 13


13

## Carregar os dados

Com os arquivos em mãos, é possível carregá-los para que os dados contidos possam ser utilizados para a criação do modelo.

Nessa etapa, os dados são carregados a partir de arquivos .csv contendo as informações dos acórdãos. Esses arquivos .csv já foram pre-processados no notebook Data_Preprocessing.ipynb, que se encontra na pasta Projeto do Google Drive. No pre-processamento as classificações dos ramos do direito de cada acórdão são mapeadas para valores numéricos que o BERT consiga entender. Esse mapeamento segue o seguinte conjunto de chaves e valores:
- Direito Penal (Direito Processual Penal) &rarr; 0
- Direito Administrativo (Licitações, Contratos Administrativos, Servidores, Desapropriação, Tribunal de Contas, Improbidade, etc.) &rarr; 1
- Direito Tributário/Direito Financeiro &rarr; 2
- Direito Civil (Direito Comercial/Direito de Família) &rarr; 3
- Direito Previdenciário &rarr; 4
- Direito do Trabalho &rarr; 5
- Direito Processual Civil &rarr; 6
- Direito Eleitoral &rarr; 7
- Direito do Consumidor &rarr; 8
- Direito Internacional (Público ou Privado) &rarr; 9
- Direito Militar &rarr; 10
- Direito Econômico (Direito concorrencial e Agências Reguladoras Setoriais, Intervenção no Domínio Econômico) &rarr; 11
- Direito Ambiental &rarr; 12

Há dois conjunto de dados a serem carregados: treinamento e validação.

In [ ]:
# Read the training dataset from .csv file
documents = pd.read_csv(CONSTANTS['TRAINING_DATASET'])
documents

,Unnamed: 0,cod_acordao,ramo,tipo_acordao,cabecalho,ementa,decisao,indexacao,somente_ementa,indicacao_exclusiva_ementa_voto,expressoes_chave
0,2470,MS 25697,1,MS,MS 25697 / DF - DISTRITO FEDERAL MANDADO DE SE...,EMENTA: ADMINISTRATIVO. APOSENTADORIA DE SERVI...,"O Tribunal, por unanimidade e nos termos do vo...","['AUSÊNCIA', 'DECADÊNCIA ADMINISTRATIVA', 'ATO...",Sim,Sim,administrativo
1,2106,ADI 2810,1,ADI,ADI 2810 / RS - RIO GRANDE DO SUL AÇÃO DIRETA ...,Ementa: Processo constitucional. Ação direta d...,"O Tribunal, por unanimidade e nos termos do vo...","['OCORRÊNCIA', 'CASO CONCRETO', 'AUMENTO', 'RE...",Sim,Não,servidores públicos
2,3255,ADI 1175,1,ADI,ADI 1175 / DF - DISTRITO FEDERAL AÇÃO DIRETA D...,TRIBUNAL DE CONTAS - CONTROLE. Surge harmônico...,Após os votos dos Senhores Ministros Carlos Ve...,"['CONSTITUCIONALIDADE', 'DISPOSITIVO', 'LEI OR...",Sim,Sim,Tribunal de contas
3,3956,HC 94398,0,HC,HC 94398 / RJ - RIO DE JANEIRO HABEAS CORPUS R...,EMENTA: HABEAS CORPUS. PENAL E PROCESSUAL PENA...,"A Turma, à unanimidade, denegou a ordem de hab...",['VIDE EMENTA'],Sim,Sim,habeas corpus
4,4731,RHC 96093,10,RHC,RHC 96093 / PA - PARÁ RECURSO EM HABEAS CORPUS...,EMENTA: RECURSO ORDINÁRIO EM HABEAS CORPUS. TR...,"A Turma, à unanimidade, negou provimento ao re...",['VIDE EMENTA'],Não,Não,inquérito policial militar
...,...,...,...,...,...,...,...,...,...,...,...
3861,2796,RE 587008,2,RE,RE 587008 / SP - SÃO PAULO RECURSO EXTRAORDINÁ...,EMENTA Recurso extraordinário – Emenda Constit...,"O Tribunal, por unanimidade e nos termos do vo...","['PRINCÍPIO DA ANTERIORIDADE NONAGESIMAL', 'IN...",Sim,Não,Contribuição Social sobre o Lucro
3862,5173,MS 21896,1,MS,MS 21896 / PB - PARAÍBA MANDADO DE SEGURANÇA R...,EMENTA: MANDADO DE SEGURANÇA. DECRETO HOMOLOGA...,Após o voto do Senhor Ministro Carlos Velloso ...,"['DECRETO PRESIDENCIAL', 'HOMOLOGAÇÃO', 'DEMAR...",Sim,Não,DEMARCAÇÃO ADMINISTRATIVA
3863,4664,RHC 80741,0,RHC,RHC 80741 / PA - PARÁ RECURSO EM HABEAS CORPUS...,EMENTA: RECURSO ORDINÁRIO EM HABEAS-CORPUS. PR...,"Por maioria, a Turma, negou provimento ao recu...","['PP0021', 'PRISÃO PREVENTIVA', 'PRAZO', 'EXCE...",Sim,Não,Código de processo penal
3864,2049,RE 423560,1,RE,RE 423560 / MG - MINAS GERAIS RECURSO EXTRAORD...,Ementa: DIREITO CONSTITUCIONAL E ADMINISTRATIV...,"A Turma, por unanimidade, deu provimento ao re...",['VIDE EMENTA'],Sim,Sim,"""direito administrativo""; ""lei orgânica do mun..."


In [ ]:
# Read the validation dataset from .csv file
documents_val = pd.read_csv(CONSTANTS['VALIDATION_DATASET'])
documents_val

,Unnamed: 0,cod_acordao,ramo,tipo_acordao,cabecalho,ementa,decisao,indexacao,somente_ementa,indicacao_exclusiva_ementa_voto,expressoes_chave
0,5449,RE 250948,4,RE,RE 250948 / RS - RIO GRANDE DO SUL RECURSO EXT...,EMENTA: - Recurso extraordinário. Administrati...,"Por unanimidade, a Turma não conheceu do recur...","['DESEMBARGADOR', 'PREENCHIMENTO', 'REQUISITOS...",Sim,Sim,Aposentadoria
1,4453,HC 122579,0,HC,HC 122579 / SP - SÃO PAULO HABEAS CORPUS Relat...,Ementa: HABEAS CORPUS. DIREITO PROCESSUAL PENA...,"Por maioria de votos, a Turma julgou extinto o...","['VIDE EMENTA', 'VOTO VENCIDO', 'MIN', 'MARCO ...",Sim,Sim,habeas corpus
2,1600,EXT 1456,9,EXT,Ext 1456 / DF - DISTRITO FEDERAL EXTRADIÇÃO Re...,Ementa: EXTRADIÇÃO INSTRUTÓRIA. TRATADO DE EXT...,"A Turma deferiu o pedido de extradição, nos te...",['VIDE EMENTA'],Sim,Sim,extradição
3,783,HC 147483,0,HC,HC 147483 / DF - DISTRITO FEDERAL HABEAS CORPU...,Ementa: HABEAS CORPUS CONTRA DECISÃO MONOCRÁTI...,"A Turma, por maioria, não conheceu da impetraç...",['VIDE EMENTA'],Sim,Não,constrangimento legal
4,2526,RHC 117982,0,RHC,RHC 117982 / ES - ESPÍRITO SANTO RECURSO ORDIN...,Ementa: RECURSO ORDINÁRIO EM HABEAS CORPUS. PE...,"A Turma, por unanimidade, negou provimento ao ...",['VIDE EMENTA'],Sim,Sim,habeas corpus
...,...,...,...,...,...,...,...,...,...,...,...
824,3602,RHC 122182,0,RHC,RHC 122182 / SP - SÃO PAULO RECURSO ORDINÁRIO ...,EMENTA: PENAL E PROCESSUAL PENAL. RECURSO ORDI...,"Por maioria de votos, a Turma negou provimento...","['VIDE EMENTA', 'VOTO VENCIDO', 'MIN', 'MARCO ...",Sim,Sim,PENAL
825,3025,MS 29323,1,MS,MS 29323 / DF - DISTRITO FEDERAL MANDADO DE SE...,Ementa: CONSTITUCIONAL E ADMINISTRATIVO. MANDA...,"A Turma, por maioria, denegou a segurança e re...","['FUNDAMENTAÇÃO COMPLEMENTAR', 'MIN', 'ROSA WE...",Sim,Sim,mandado de segurança
826,3879,ADI 1756,1,ADI,ADI 1756 / MA - MARANHÃO AÇÃO DIRETA DE INCONS...,Ementa: DIREITO CONSTITUCIONAL. ADI. VINCULAÇÃ...,"O Tribunal, por unanimidade e nos termos do vo...","['AUSÊNCIA', 'ALTERAÇÃO', 'PARÂMETRO DE CONTRO...",Sim,Não,remunerações dos membros do Ministério Público...
827,4146,PET 7792,0,PET,Pet 7792 / DF - DISTRITO FEDERAL PETIÇÃO Relat...,Ementa: PETIÇÃO. AGRAVO REGIMENTAL. DENÚNCIA. ...,"A Turma, por votação unânime, negou provimento...",['VIDE EMENTA'],Sim,Sim,ação penal


## Definição dos conjunto de dados de acordo com o tamanho da ementa

O Bert tem uma limitação de lidar apenas com textos de, no máximo, 512 tokens. Portanto, uma forma de contornar a situação seria extrair parte dos termos da porção inicial do texto, e o restante da porção final. Dessa forma, o texto final incluiria o contexto da introdução e conclusão da ementa original.

Uma imagem que explica o método que será utilizado:

![long-sequences-bert](https://drive.google.com/uc?export=view&id=1MFofvPPQwfHO1GogvAEF5U300PET6nZw)

No caso, o exemplo mostra uma divisão igual entre os termos extraídos da introdução e conclusão da ementa. No entanto, o modelo também pode preferir extrair mais/menos termos da introdução. Isso será controlado pelo hiperparâmetro `INTRODUCTION_WORDS`.

In [ ]:
def get_words(sentence: str) -> List[str]:
    """Retrieves the words in a sentence by splitting into blank spaces.

    Args:
        sentence (str): a sentence.

    Returns:
        List[str]: list of words in sentence.
    """
    return re.findall(r'\S+', sentence)

def merge_front_back_summary(full_summary: str, max_summary_size: int) -> str:
    """Extracts the part of the front and part of the back from the summary,
    following the properties defined in Constants structure.

    Args:
        full_summary (str): the entire summary content.
        max_summary_size (int): the max number of tokens considered.

    Returns:
        str: a version of summary with the beginning of its front and the end
        of its back.
    """
    front_size = HYPERPARAMETERS['INTRODUCTION_WORDS']
    back_size = max_summary_size - front_size

    summary_words = get_words(full_summary)

    if (len(summary_words) <= max_summary_size):
        return full_summary

    front_summary = summary_words[:front_size]
    back_summary = summary_words[(len(summary_words) - back_size):]

    return ' '.join(front_summary).strip() + ' ' + ' '.join(back_summary).strip()

merge_front_back_summary('E M E N T A: \"HABEAS CORPUS\" - CRIME DE TORTURA ATRIBUÍDO A DELEGADO E A AGENTES POLICIAIS CIVIS - POSSIBILIDADE DE O MINISTÉRIO PÚBLICO, FUNDADO EM INVESTIGAÇÃO POR ELE PRÓPRIO PROMOVIDA, FORMULAR DENÚNCIA CONTRA REFERIDOS INTEGRANTES DA POLÍCIA CIVIL - VALIDADE JURÍDICA DESSA ATIVIDADE INVESTIGATÓRIA - CONDENAÇÃO PENAL IMPOSTA AOS POLICIAIS CIVIS - LEGITIMIDADE JURÍDICA DO PODER INVESTIGATÓRIO DO MINISTÉRIO PÚBLICO - MONOPÓLIO CONSTITUCIONAL DA TITULARIDADE DA AÇÃO PENAL PÚBLICA PELO \"PARQUET\" - TEORIA DOS PODERES IMPLÍCITOS - CASO \"McCULLOCH v. MARYLAND\" (1819) - MAGISTÉRIO DA DOUTRINA (RUI BARBOSA, JOHN MARSHALL, JOÃO BARBALHO, MARCELLO CAETANO, CASTRO NUNES, OSWALDO TRIGUEIRO, v.g.) - OUTORGA, AO MINISTÉRIO PÚBLICO, PELA PRÓPRIA CONSTITUIÇÃO DA REPÚBLICA, DO PODER DE CONTROLE EXTERNO SOBRE A ATIVIDADE POLICIAL - LIMITAÇÕES DE ORDEM JURÍDICA AO PODER INVESTIGATÓRIO DO MINISTÉRIO PÚBLICO - \"HABEAS CORPUS\" INDEFERIDO. NAS HIPÓTESES DE AÇÃO PENAL PÚBLICA, O INQUÉRITO POLICIAL, QUE CONSTITUI UM DOS DIVERSOS INSTRUMENTOS ESTATAIS DE INVESTIGAÇÃO PENAL, TEM POR DESTINATÁRIO PRECÍPUO O MINISTÉRIO PÚBLICO. - O inquérito policial qualifica-se como procedimento administrativo, de caráter pré-processual, ordinariamente vocacionado a subsidiar, nos casos de infrações perseguíveis mediante ação penal de iniciativa pública, a atuação persecutória do Ministério Público, que é o verdadeiro destinatário dos elementos que compõem a \"informatio delicti\". Precedentes. - A investigação penal, quando realizada por organismos policiais, será sempre dirigida por autoridade policial, a quem igualmente competirá exercer, com exclusividade, a presidência do respectivo inquérito. - A outorga constitucional de funções de polícia judiciária à instituição policial não impede nem exclui a possibilidade de o Ministério Público, que é o \"dominus litis\", determinar a abertura de inquéritos policiais, requisitar esclarecimentos e diligências investigatórias, estar presente e acompanhar, junto a órgãos e agentes policiais, quaisquer atos de investigação penal, mesmo aqueles sob regime de sigilo, sem prejuízo de outras medidas que lhe pareçam indispensáveis à formação da sua \"opinio delicti\", sendo-lhe vedado, no entanto, assumir a presidência do inquérito policial, que traduz atribuição privativa da autoridade policial. Precedentes. A ACUSAÇÃO PENAL, PARA SER FORMULADA, NÃO DEPENDE, NECESSARIAMENTE, DE PRÉVIA INSTAURAÇÃO DE INQUÉRITO POLICIAL. - Ainda que inexista qualquer investigação penal promovida pela Polícia Judiciária, o Ministério Público, mesmo assim, pode fazer instaurar, validamente, a pertinente \"persecutio criminis in judicio\", desde que disponha, para tanto, de elementos mínimos de informação, fundados em base empírica idônea, que o habilitem a deduzir, perante juízes e Tribunais, a acusação penal. Doutrina. Precedentes. A QUESTÃO DA CLÁUSULA CONSTITUCIONAL DE EXCLUSIVIDADE E A ATIVIDADE INVESTIGATÓRIA. - A cláusula de exclusividade inscrita no art. 144, § 1º, inciso IV, da Constituição da República - que não inibe a atividade de investigação criminal do Ministério Público - tem por única finalidade conferir à Polícia Federal, dentre os diversos organismos policiais que compõem o aparato repressivo da União Federal (polícia federal, polícia rodoviária federal e polícia ferroviária federal), primazia investigatória na apuração dos crimes previstos no próprio texto da Lei Fundamental ou, ainda, em tratados ou convenções internacionais. - Incumbe, à Polícia Civil dos Estados-membros e do Distrito Federal, ressalvada a competência da União Federal e excetuada a apuração dos crimes militares, a função de proceder à investigação dos ilícitos penais (crimes e contravenções), sem prejuízo do poder investigatório de que dispõe, como atividade subsidiária, o Ministério Público. - Função de polícia judiciária e função de investigação penal: uma distinção conceitual relevante, que também justifica o reconhecimento, ao Ministério Público, do poder investigatório em matéria penal. Doutrina. É PLENA A LEGITIMIDADE CONSTITUCIONAL DO PODER DE INVESTIGAR DO MINISTÉRIO PÚBLICO, POIS OS ORGANISMOS POLICIAIS (EMBORA DETENTORES DA FUNÇÃO DE POLÍCIA JUDICIÁRIA) NÃO TÊM, NO SISTEMA JURÍDICO BRASILEIRO, O MONOPÓLIO DA COMPETÊNCIA PENAL INVESTIGATÓRIA. - O poder de investigar compõe, em sede penal, o complexo de funções institucionais do Ministério Público, que dispõe, na condição de \"dominus litis\" e, também, como expressão de sua competência para exercer o controle externo da atividade policial, da atribuição de fazer instaurar, ainda que em caráter subsidiário, mas por autoridade própria e sob sua direção, procedimentos de investigação penal destinados a viabilizar a obtenção de dados informativos, de subsídios probatórios e de elementos de convicção que lhe permitam formar a \"opinio delicti\", em ordem a propiciar eventual ajuizamento da ação penal de iniciativa pública. Doutrina. Precedentes: RE 535.478/SC, Rel. Min. ELLEN GRACIE - HC 91.661/PE, Rel. Min. ELLEN GRACIE - HC 85.419/RJ, Rel. Min. CELSO DE MELLO - HC 89.837/DF, Rel. Min. CELSO DE MELLO. CONTROLE JURISDICIONAL DA ATIVIDADE INVESTIGATÓRIA DOS MEMBROS DO MINISTÉRIO PÚBLICO: OPONIBILIDADE, A ESTES, DO SISTEMA DE DIREITOS E GARANTIAS INDIVIDUAIS, QUANDO EXERCIDO, PELO \"PARQUET\", O PODER DE INVESTIGAÇÃO PENAL. - O Ministério Público, sem prejuízo da fiscalização intra--orgânica e daquela desempenhada pelo Conselho Nacional do Ministério Público, está permanentemente sujeito ao controle jurisdicional dos atos que pratique no âmbito das investigações penais que promova \"ex propria auctoritate\", não podendo, dentre outras limitações de ordem jurídica, desrespeitar o direito do investigado ao silêncio (\"nemo tenetur se detegere\"), nem lhe ordenar a condução coercitiva, nem constrangê-lo a produzir prova contra si próprio, nem lhe recusar o conhecimento das razões motivadoras do procedimento investigatório, nem submetê-lo a medidas sujeitas à reserva constitucional de jurisdição, nem impedi-lo de fazer-se acompanhar de Advogado, nem impor, a este, indevidas restrições ao regular desempenho de suas prerrogativas profissionais (Lei nº 8.906/94, art. 7º, v.g.). - O procedimento investigatório instaurado pelo Ministério Público deverá conter todas as peças, termos de declarações ou depoimentos, laudos periciais e demais subsídios probatórios coligidos no curso da investigação, não podendo, o \"Parquet\", sonegar, selecionar ou deixar de juntar, aos autos, quaisquer desses elementos de informação, cujo conteúdo, por referir-se ao objeto da apuração penal, deve ser tornado acessível tanto à pessoa sob investigação quanto ao seu Advogado. - O regime de sigilo, sempre excepcional, eventualmente prevalecente no contexto de investigação penal promovida pelo Ministério Público, não se revelará oponível ao investigado e ao Advogado por este constituído, que terão direito de acesso - considerado o princípio da comunhão das provas - a todos os elementos de informação que já tenham sido formalmente incorporados aos autos do respectivo procedimento investigatório.', HYPERPARAMETERS['MAX_NUMBER_TOKENS'])

'E M E N T A: "HABEAS CORPUS" - CRIME DE TORTURA ATRIBUÍDO A DELEGADO E A AGENTES POLICIAIS CIVIS - POSSIBILIDADE DE O MINISTÉRIO PÚBLICO, FUNDADO EM INVESTIGAÇÃO POR ELE PRÓPRIO PROMOVIDA, FORMULAR DENÚNCIA CONTRA REFERIDOS INTEGRANTES DA POLÍCIA CIVIL - VALIDADE JURÍDICA DESSA ATIVIDADE INVESTIGATÓRIA - CONDENAÇÃO PENAL IMPOSTA AOS POLICIAIS CIVIS - LEGITIMIDADE JURÍDICA DO PODER INVESTIGATÓRIO DO MINISTÉRIO PÚBLICO - MONOPÓLIO CONSTITUCIONAL DA TITULARIDADE DA AÇÃO PENAL PÚBLICA PELO "PARQUET" - TEORIA DOS PODERES IMPLÍCITOS - CASO "McCULLOCH v. MARYLAND" (1819) - MAGISTÉRIO DA DOUTRINA (RUI BARBOSA, JOHN MARSHALL, JOÃO BARBALHO, MARCELLO CAETANO, CASTRO NUNES, OSWALDO TRIGUEIRO, v.g.) - OUTORGA, AO MINISTÉRIO PÚBLICO, PELA PRÓPRIA CONSTITUIÇÃO DA REPÚBLICA, DO PODER DE CONTROLE EXTERNO SOBRE A ATIVIDADE POLICIAL - LIMITAÇÕES DE ORDEM JURÍDICA AO PODER INVESTIGATÓRIO DO MINISTÉRIO PÚBLICO - "HABEAS CORPUS" INDEFERIDO. NAS HIPÓTESES DE AÇÃO PENAL PÚBLICA, O INQUÉRITO POLICIAL, QUE CO

In [ ]:
# Training dataset: Change column containing summary to considers front and back of it
documents['ementa'] = documents['ementa'].apply(lambda summary: merge_front_back_summary(summary, HYPERPARAMETERS['MAX_NUMBER_TOKENS']))
documents = documents.iloc[: , 1:]
documents.head()

,cod_acordao,ramo,tipo_acordao,cabecalho,ementa,decisao,indexacao,somente_ementa,indicacao_exclusiva_ementa_voto,expressoes_chave
0,MS 25697,1,MS,MS 25697 / DF - DISTRITO FEDERAL MANDADO DE SE...,EMENTA: ADMINISTRATIVO. APOSENTADORIA DE SERVI...,"O Tribunal, por unanimidade e nos termos do vo...","['AUSÊNCIA', 'DECADÊNCIA ADMINISTRATIVA', 'ATO...",Sim,Sim,administrativo
1,ADI 2810,1,ADI,ADI 2810 / RS - RIO GRANDE DO SUL AÇÃO DIRETA ...,Ementa: Processo constitucional. Ação direta d...,"O Tribunal, por unanimidade e nos termos do vo...","['OCORRÊNCIA', 'CASO CONCRETO', 'AUMENTO', 'RE...",Sim,Não,servidores públicos
2,ADI 1175,1,ADI,ADI 1175 / DF - DISTRITO FEDERAL AÇÃO DIRETA D...,TRIBUNAL DE CONTAS - CONTROLE. Surge harmônico...,Após os votos dos Senhores Ministros Carlos Ve...,"['CONSTITUCIONALIDADE', 'DISPOSITIVO', 'LEI OR...",Sim,Sim,Tribunal de contas
3,HC 94398,0,HC,HC 94398 / RJ - RIO DE JANEIRO HABEAS CORPUS R...,EMENTA: HABEAS CORPUS. PENAL E PROCESSUAL PENA...,"A Turma, à unanimidade, denegou a ordem de hab...",['VIDE EMENTA'],Sim,Sim,habeas corpus
4,RHC 96093,10,RHC,RHC 96093 / PA - PARÁ RECURSO EM HABEAS CORPUS...,EMENTA: RECURSO ORDINÁRIO EM HABEAS CORPUS. TR...,"A Turma, à unanimidade, negou provimento ao re...",['VIDE EMENTA'],Não,Não,inquérito policial militar


In [ ]:
# Validation dataset: Change column containing summary to considers front and back of it
documents_val['ementa'] = documents_val['ementa'].apply(lambda summary: merge_front_back_summary(summary, HYPERPARAMETERS['MAX_NUMBER_TOKENS']))
documents_val = documents_val.iloc[: , 1:]
documents_val.head()

,cod_acordao,ramo,tipo_acordao,cabecalho,ementa,decisao,indexacao,somente_ementa,indicacao_exclusiva_ementa_voto,expressoes_chave
0,RE 250948,4,RE,RE 250948 / RS - RIO GRANDE DO SUL RECURSO EXT...,EMENTA: - Recurso extraordinário. Administrati...,"Por unanimidade, a Turma não conheceu do recur...","['DESEMBARGADOR', 'PREENCHIMENTO', 'REQUISITOS...",Sim,Sim,Aposentadoria
1,HC 122579,0,HC,HC 122579 / SP - SÃO PAULO HABEAS CORPUS Relat...,Ementa: HABEAS CORPUS. DIREITO PROCESSUAL PENA...,"Por maioria de votos, a Turma julgou extinto o...","['VIDE EMENTA', 'VOTO VENCIDO', 'MIN', 'MARCO ...",Sim,Sim,habeas corpus
2,EXT 1456,9,EXT,Ext 1456 / DF - DISTRITO FEDERAL EXTRADIÇÃO Re...,Ementa: EXTRADIÇÃO INSTRUTÓRIA. TRATADO DE EXT...,"A Turma deferiu o pedido de extradição, nos te...",['VIDE EMENTA'],Sim,Sim,extradição
3,HC 147483,0,HC,HC 147483 / DF - DISTRITO FEDERAL HABEAS CORPU...,Ementa: HABEAS CORPUS CONTRA DECISÃO MONOCRÁTI...,"A Turma, por maioria, não conheceu da impetraç...",['VIDE EMENTA'],Sim,Não,constrangimento legal
4,RHC 117982,0,RHC,RHC 117982 / ES - ESPÍRITO SANTO RECURSO ORDIN...,Ementa: RECURSO ORDINÁRIO EM HABEAS CORPUS. PE...,"A Turma, por unanimidade, negou provimento ao ...",['VIDE EMENTA'],Sim,Sim,habeas corpus


## Preparação para o treinamento do modelo

Depois de ter os dados organizados, a melhor forma de treinar o modelo é antecipar uma preparação dos dados. Dessa forma, geralmente cria-se um Dataset para que o modelo possa consumir os dados facilmente.

O Dataset auxilia a modularizar o código utilizado para treinar o modelo. Dessa forma, as rotinas para manter uma coleção de dados utilizada para o modelo podem ser isoladas no Dataset. O Dataset basicamente compreende amostras de dados com seus respectivos rótulos (saída do modelo).

In [ ]:
class LawDocumentDataset(Dataset):
  def __init__(self, dataframe: pd.DataFrame, tokenizer: BertTokenizer, max_token_length: int=512):
    self.dataframe = dataframe
    self.tokenizer = tokenizer
    self.max_token_length = max_token_length

  def __len__(self):
    return len(self.dataframe)

  def __getitem__(self, index: int):
    row = self.dataframe.iloc[index]
    summary_document = row.ementa
    law_branch_id = row.ramo

    encoding = self.tokenizer.encode_plus(
      summary_document,
      add_special_tokens=True,          # Add `[CLS]` and `[SEP]`
      max_length=self.max_token_length,
      return_token_type_ids=False,
      padding="max_length",
      truncation=True,                  # Truncate encoding to the max length
      return_attention_mask=True,       # Return attention mask
      return_tensors="pt"               # Return PyTorch tensor
    )

    labels = np.eye(HYPERPARAMETERS['NUMBER_OF_BRANCHES'])[law_branch_id]  # Return a list with zeros, except for index law_branch_id that assumes one

    return dict(
        summary_document=summary_document,
        input_ids=encoding["input_ids"].flatten(),
        attention_mask=encoding["attention_mask"].flatten(),
        labels=torch.FloatTensor(labels)
    )

In [ ]:
tokenizer = BertTokenizer.from_pretrained(CONSTANTS['BERT_MODEL_NAME'])
tokenizer

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/155 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

PreTrainedTokenizerFast(name_or_path='neuralmind/bert-large-portuguese-cased', vocab_size=29794, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
train_dataset = LawDocumentDataset(documents, tokenizer, HYPERPARAMETERS['MAX_NUMBER_TOKENS'])
validation_dataset = LawDocumentDataset(documents_val, tokenizer, HYPERPARAMETERS['MAX_NUMBER_TOKENS'])

## Criando o modelo

Depois de ter toda a preparação dos dados, o modelo pode então começar a ser treinado. Para isso, utilizar LightiningModule pode auxiliar durante o processo. O código normalmente utilizado para o treinamento de uma rede neural usando Pytorch pode ser compactado por meio do LightningModule. LightningModule permite que o treinamento do modelo esteja disposto de uma forma organizada no código, também prevenindo que chamadas utilizando `.cuda()` ou `.to()` sejam realizadas. A própria classe se responsabiliza para controlar quais tensores devem abrigar cálculos dentro da GPU.

Basicamente, o processo utilizado pelo LightningModule para o treinamento do modelo é:

```
for epoch in range(num_epochs):
    # Training phase
    for batch in train_loader:
        for each entry in batch, run forward
        run training_step
        calculate loss & metrics
    
    # Validation phase
    for batch in validation_loader:
        run validation_step
        calculate loss & metrics
    
    # Test step
    for batch in test_loader:
        run test_step
        calculate loss & metrics
```

In [ ]:
class LawDocumentClassifier(pl.LightningModule):
    
    def __init__(self, number_classes: int, steps_per_epoch: int=None, epochs: int=None, learning_rate: float=2e-5, weight_decay: float=0.01, warm_up_proportion: float=0.1):
        super().__init__()
        
        self.model = BertForSequenceClassification.from_pretrained(
            "neuralmind/bert-large-portuguese-cased",
            num_labels=number_classes,                      # The number of output labels--2 for binary classification
            output_attentions=False,                        # Returns attention weights
            output_hidden_states=False                      # Returns all hidden states
        )
        self.steps_per_epoch = steps_per_epoch
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.warm_up_proportion = warm_up_proportion
        self.weight_decay = weight_decay
        
    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids,
                            attention_mask=attention_mask,
                            labels=labels,
                            return_dict=True)
        
        return output.loss, output.logits
        
    def training_step(self, batch, batch_index):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        
        loss, outputs = self(input_ids, attention_mask, labels)
        
        self.log("train_loss", loss, prog_bar=True, logger=True)
        
        return {"loss": loss, "predictions": outputs, "labels": labels}

    def compute_metrics(self, eval_pred):
        logits, labels = eval_pred
        
        classification_predictions = self.convert_to_classification_labels(logits)
        classification_labels = self.convert_to_classification_labels(labels)

        metrics = {
            "validation_accuracy": accuracy_score(classification_labels, classification_predictions),
            "validation_balanced_accuracy": balanced_accuracy_score(classification_labels, classification_predictions),
            "validation_precision": precision_score(classification_labels, classification_predictions, average='weighted'),
            "validation_recall": recall_score(classification_labels, classification_predictions, average='weighted'),
            "validation_f1": f1_score(classification_labels, classification_predictions, average='weighted'),
            "validation_cohen_kappa": cohen_kappa_score(classification_labels, classification_predictions),
            "validation_matthews": matthews_corrcoef(classification_labels, classification_predictions)
        }

        return metrics
            
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        warmup_steps = self.steps_per_epoch * self.warm_up_proportion
        total_steps = self.steps_per_epoch * self.epochs - warmup_steps

        scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
        
        return (optimizer, scheduler)

    def convert_to_classification_labels(self, classifications):
        formatted_classifications = []

        for classification in classifications:
            formatted_classifications.append(np.argmax(classification).flatten())

        return formatted_classifications

In [ ]:
model = LawDocumentClassifier(
    HYPERPARAMETERS['NUMBER_OF_BRANCHES'],
    steps_per_epoch=len(documents) // HYPERPARAMETERS['BATCH_SIZE'],
    epochs=HYPERPARAMETERS['EPOCHS'],
    learning_rate=HYPERPARAMETERS['LEARNING_RATE'],
    weight_decay=HYPERPARAMETERS['WEIGHT_DECAY'],
    warm_up_proportion=HYPERPARAMETERS['WARM_UP_PROPORTION']
)

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-large-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from th

## Treinando o modelo

Basicamente, o processo utilizado pelo LightningModule para o treinamento do modelo é:

```
for epoch in range(num_epochs):
    # Training phase
    for batch in train_loader:
        for each entry in batch, run forward
        run training_step
        calculate loss & metrics
    
    # Validation phase
    for batch in validation_loader:
        run validation_step
        calculate loss & metrics
    
    # Test step
    for batch in test_loader:
        run test_step
        calculate loss & metrics
```

O processo para treinamento do modelo será executado pela API de [Trainer](https://huggingface.co/transformers/main_classes/trainer.html) do HuggingFace. Nesse sentido, alguns parâmetros são definidos durante a criação do objeto `Trainer` para a criação da estratégia de treinamento.

In [ ]:
training_args = TrainingArguments(
    "/content/drive/My Drive/MAC499 - Kaique e Yurick/Projeto/trainer_output",
    num_train_epochs=HYPERPARAMETERS['EPOCHS'],
    evaluation_strategy='epoch',
    per_device_train_batch_size=HYPERPARAMETERS['BATCH_SIZE'],
    logging_steps=30
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=model.compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    optimizers=model.configure_optimizers()
)
trainer.train()

***** Running training *****
  Num examples = 3866
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 9665


Epoch,Training Loss,Validation Loss,Validation Accuracy,Validation Precision,Validation Recall,Validation F1
1,0.118000,0.094661,0.802171,0.774728,0.802171,0.774814
2,0.125900,0.078145,0.835947,0.839844,0.835947,0.824025
3,0.075200,0.084746,0.832328,0.831785,0.832328,0.827697
4,0.033000,0.093404,0.821472,0.821004,0.821472,0.817800
5,0.049800,0.094311,0.828709,0.828048,0.828709,0.824539


Saving model checkpoint to /content/drive/My Drive/MAC499 - Kaique e Yurick/Projeto/trainer_output/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/My Drive/MAC499 - Kaique e Yurick/Projeto/trainer_output/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/My Drive/MAC499 - Kaique e Yurick/Projeto/trainer_output/checkpoint-1500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 829
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /content/drive/My Drive/MAC499 - Kaique e Yurick/Pro

TrainOutput(global_step=9665, training_loss=0.07525187440937317, metrics={'train_runtime': 18393.8804, 'train_samples_per_second': 1.051, 'train_steps_per_second': 0.525, 'total_flos': 0.0, 'train_loss': 0.07525187440937317, 'epoch': 5.0})

## Análise de métricas

Após a realização do treinamento do modelo, o ideal é extrair as métricas que foram coletadas durante os processos de treinamento e validação em cada uma das épocas, para que então os melhores valores de hiperparâmetros sejam extraídos.

A API `Trainer` de HuggingFace possui a capacidade de calcular automaticamente as métricas configuradas e coletar os valores durante as épocas. Esses valores serão extraídos para análises, como parte do processo de fine tuning dos hiperparâmetros.

In [ ]:
validation_metrics = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 829
  Batch size = 8


In [ ]:
validation_metrics

{'epoch': 5.0,
 'eval_loss': 0.09431097656488419,
 'eval_runtime': 180.0057,
 'eval_samples_per_second': 4.605,
 'eval_steps_per_second': 0.578,
 'eval_validation_accuracy': 0.8287092882991556,
 'eval_validation_f1': 0.8245385972421762,
 'eval_validation_precision': 0.828047839970979,
 'eval_validation_recall': 0.8287092882991556}

In [ ]:
# to save the model, uncomment the next lines

# filename = 'front_back-{introduction_words}-{learning_rate}-{weight_decay}-{warmup_proportion}.bin'.format(introduction_words=HYPERPARAMETERS['INTRODUCTION_WORDS'], learning_rate=HYPERPARAMETERS['LEARNING_RATE'], weight_decay=HYPERPARAMETERS['WEIGHT_DECAY'], warmup_proportion=HYPERPARAMETERS['INTRODUCTION_WORDS'])
# torch.save(model.state_dict(), '/content/drive/My Drive/MAC499 - Kaique e Yurick/Projeto/'.format())

## Referências

- https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
- https://arxiv.org/pdf/1711.05101.pdf
- https://arxiv.org/pdf/1905.05583.pdf
- https://arxiv.org/pdf/1810.04805.pdf
- https://www.fast.ai/2018/07/02/adam-weight-decay/
- Valores para taxa de decaimento: https://openreview.net/pdf?id=Syx4wnEtvH

Cross-entropy Loss
Optimizer = AdamW (valores podem ser retirados [daqui](https://arxiv.org/pdf/1905.05583.pdf))
Scheduler = Cosine sem reset (Appendix C do [paper](https://arxiv.org/pdf/1711.05101.pdf))